# Benchmarking: single node scaling

We can benchmark the allocation algorithms and determine their behavior with respect
to the size of the fleet and the number of charging posts. We simply need to create
the fleets and charging posts and then run the algorithms via :py:mod:`timeit`, a
timing module form the python standard library.

Lets first define the sizes for which we will run benchmarks, ensuring the size of the
fleet and the number of posts are not too different.

In [1]:
from functools import partial
from typing import Callable, Mapping, Optional

import numpy as np
import pandas as pd
import statsmodels.api as sm

In [2]:
from patsy import dmatrices

import evosim


def benchmark_set(
    n=(10, 50),
    exps=(1, 10, 100, 1000),
    algos=("greedy", "random"),
    ratios=(0.5, 1, 1.5),
):
    ns = np.array(n)
    ns = np.concatenate([ns * u for u in exps])
    result = pd.DataFrame(
        dict(
            fleet=np.concatenate([ns] * (len(ratios) * len(algos))),
            infrastructure=np.concatenate(
                [np.round(ns * r).astype(int) for a in algos for r in ratios]
            ),
            ratios=np.concatenate([[r] * len(ns) for a in algos for r in ratios]),
            algorithm=np.concatenate([[a] * len(ns) for a in algos for r in ratios]),
        )
    )
    result["ratio"] = (result.infrastructure / result.fleet).astype("category")
    return result[["algorithm", "fleet", "infrastructure", "ratio"]]


benchmarks = benchmark_set(n=(10, 25, 50, 75), exps=(10, 100))
benchmarks.loc[:10]

,algorithm,fleet,infrastructure,ratio
0,greedy,100,50,0.5
1,greedy,250,125,0.5
2,greedy,500,250,0.5
3,greedy,750,375,0.5
4,greedy,1000,500,0.5
5,greedy,2500,1250,0.5
6,greedy,5000,2500,0.5
7,greedy,7500,3750,0.5
8,greedy,100,100,1.0
9,greedy,250,250,1.0


Finally, we define a function to run the given algorithm for a given setup:

In [3]:
def run_benchmark(
    row: pd.Series,
    algos: Optional[Mapping] = None,
    matcher: Optional[Callable] = None,
    repetitions: int = 10,
    **kwargs,
) -> float:
    from timeit import timeit
    from evosim.matchers import socket_compatibility
    from evosim.allocators import random_allocator, greedy_allocator

    fleet = evosim.fleet.random_fleet(row.fleet, **kwargs)
    infrastructure = evosim.charging_posts.random_charging_posts(
        row.infrastructure, **kwargs
    )
    matcher = matcher or socket_compatibility
    algos = algos or dict(greedy=greedy_allocator, random=random_allocator)
    inputs = dict(fleet=fleet, infrastructure=infrastructure, matcher=matcher, **algos)
    return (
        timeit(
            f"{row.algorithm}(fleet, infrastructure, matcher)",
            globals=inputs,
            number=repetitions,
        )
        / repetitions
    )

We can run the benchmarks one after the other with :py:func:`pandas.DataFrame.apply`:

In [4]:
rng = np.random.default_rng(1)
benchmarks["timings"] = benchmarks.apply(partial(run_benchmark, seed=rng), axis=1)
benchmarks.sample(10, random_state=1)


y, X = dmatrices(
    "timings ~ algorithm:ratio:fleet", data=benchmarks, return_type="dataframe"
)
model = sm.OLS(y, X)
fit = model.fit()
fit.summary()

/Users/mdavezac/imperial/evosim/src/evosim/allocators.py:332: AllocationWarning: Could not allocate all vehicles: increasing ``nearest_neighbors`` might be beneficial
  warn(msg, AllocationWarning)
/Users/mdavezac/imperial/evosim/src/evosim/allocators.py:332: AllocationWarning: Could not allocate all vehicles: increasing ``nearest_neighbors`` might be beneficial
  warn(msg, AllocationWarning)
/Users/mdavezac/imperial/evosim/src/evosim/allocators.py:332: AllocationWarning: Could not allocate all vehicles: increasing ``nearest_neighbors`` might be beneficial
  warn(msg, AllocationWarning)
/Users/mdavezac/imperial/evosim/src/evosim/allocators.py:332: AllocationWarning: Could not allocate all vehicles: increasing ``nearest_neighbors`` might be beneficial
  warn(msg, AllocationWarning)
/Users/mdavezac/imperial/evosim/src/evosim/allocators.py:332: AllocationWarning: Could not allocate all vehicles: increasing ``nearest_neighbors`` might be beneficial
  warn(msg, AllocationWarning)
/Users/mda

/Users/mdavezac/imperial/evosim/src/evosim/allocators.py:332: AllocationWarning: Could not allocate all vehicles: increasing ``nearest_neighbors`` might be beneficial
  warn(msg, AllocationWarning)
/Users/mdavezac/imperial/evosim/src/evosim/allocators.py:332: AllocationWarning: Could not allocate all vehicles: increasing ``nearest_neighbors`` might be beneficial
  warn(msg, AllocationWarning)
/Users/mdavezac/imperial/evosim/src/evosim/allocators.py:332: AllocationWarning: Could not allocate all vehicles: increasing ``nearest_neighbors`` might be beneficial
  warn(msg, AllocationWarning)
/Users/mdavezac/imperial/evosim/src/evosim/allocators.py:332: AllocationWarning: Could not allocate all vehicles: increasing ``nearest_neighbors`` might be beneficial
  warn(msg, AllocationWarning)
/Users/mdavezac/imperial/evosim/src/evosim/allocators.py:332: AllocationWarning: Could not allocate all vehicles: increasing ``nearest_neighbors`` might be beneficial
  warn(msg, AllocationWarning)
/Users/mda

/Users/mdavezac/imperial/evosim/src/evosim/allocators.py:332: AllocationWarning: Could not allocate all vehicles: increasing ``nearest_neighbors`` might be beneficial
  warn(msg, AllocationWarning)
/Users/mdavezac/imperial/evosim/src/evosim/allocators.py:332: AllocationWarning: Could not allocate all vehicles: increasing ``nearest_neighbors`` might be beneficial
  warn(msg, AllocationWarning)
/Users/mdavezac/imperial/evosim/src/evosim/allocators.py:332: AllocationWarning: Could not allocate all vehicles: increasing ``nearest_neighbors`` might be beneficial
  warn(msg, AllocationWarning)
/Users/mdavezac/imperial/evosim/src/evosim/allocators.py:332: AllocationWarning: Could not allocate all vehicles: increasing ``nearest_neighbors`` might be beneficial
  warn(msg, AllocationWarning)
/Users/mdavezac/imperial/evosim/src/evosim/allocators.py:332: AllocationWarning: Could not allocate all vehicles: increasing ``nearest_neighbors`` might be beneficial
  warn(msg, AllocationWarning)
/Users/mda

/Users/mdavezac/imperial/evosim/src/evosim/allocators.py:332: AllocationWarning: Could not allocate all vehicles: increasing ``nearest_neighbors`` might be beneficial
  warn(msg, AllocationWarning)
/Users/mdavezac/imperial/evosim/src/evosim/allocators.py:332: AllocationWarning: Could not allocate all vehicles: increasing ``nearest_neighbors`` might be beneficial
  warn(msg, AllocationWarning)
/Users/mdavezac/imperial/evosim/src/evosim/allocators.py:332: AllocationWarning: Could not allocate all vehicles: increasing ``nearest_neighbors`` might be beneficial
  warn(msg, AllocationWarning)
/Users/mdavezac/imperial/evosim/src/evosim/allocators.py:332: AllocationWarning: Could not allocate all vehicles: increasing ``nearest_neighbors`` might be beneficial
  warn(msg, AllocationWarning)
/Users/mdavezac/imperial/evosim/src/evosim/allocators.py:332: AllocationWarning: Could not allocate all vehicles: increasing ``nearest_neighbors`` might be beneficial
  warn(msg, AllocationWarning)
/Users/mda

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                timings   R-squared:                       0.990
Model:                            OLS   Adj. R-squared:                  0.989
Method:                 Least Squares   F-statistic:                     708.2
Date:                Mon, 30 Nov 2020   Prob (F-statistic):           9.37e-40
Time:                        21:15:41   Log-Likelihood:                 1.2952
No. Observations:                  48   AIC:                             11.41
Df Residuals:                      41   BIC:                             24.51
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
======================================================================================================
                                         coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------------------------
Intercept                              0.0989      0.049      2.024      0.050       0.000       0.198
algorithm[greedy]:ratio[0.5]:fleet     0.0006   2.86e-05     19.566      0.000       0.001       0.001
algorithm[random]:ratio[0.5]:fleet     0.0008   2.86e-05     28.926      0.000       0.001       0.001
algorithm[greedy]:ratio[1.0]:fleet     0.0010   2.86e-05     34.406      0.000       0.001       0.001
algorithm[random]:ratio[1.0]:fleet     0.0014   2.86e-05     49.979      0.000       0.001       0.001
algorithm[greedy]:ratio[1.5]:fleet     0.0009   2.86e-05     30.302      0.000       0.001       0.001
algorithm[random]:ratio[1.5]:fleet     0.0002   2.86e-05      7.851      0.000       0.000       0.000
==============================================================================
Omnibus:                       40.695   Durbin-Watson:                   2.439
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              197.845
Skew:                           1.988   Prob(JB):                     1.09e-43
Kurtosis:                      12.117   Cond. No.                     1.81e+03
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 1.81e+03. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [5]:
def plotting(benchmarks):
    from bokeh.plotting import figure, output_file, output_notebook, show
    from bokeh import palettes

    output_notebook()
    p = figure()
    colors = dict(greedy=palettes.Blues9, random=palettes.Reds9)
    for algorithm in set(benchmarks.algorithm):
        algodata = benchmarks[benchmarks.algorithm == algorithm]
        for color, ratio in zip(colors[algorithm], sorted(set(algodata.ratio))):
            n = np.round(algodata.fleet * ratio)
            data = algodata[n == algodata.infrastructure]
            p.scatter(
                data.fleet,
                data.timings,
                radius=60,
                color=color,
                line_color=None,
                legend_label=f"{algorithm}: {ratio}",
            )
            p.line(
                data.fleet, data.prediction, line_color=color,
            )

    show(p)


predictions = benchmarks.copy(deep=False)
predictions["prediction"] = fit.predict()
plotting(predictions)

Loading BokehJS ...